# Module 4: Convolutional Neural Networks (CNNs)

---
## How Does a Computer See?

### 🧠 Brain Analogy
When you look at a cat photo, your brain doesn't process every pixel separately:
1. **V1** → detects edges at different angles
2. **V2, V4** → combines edges into textures and shapes (fur, whiskers)
3. **Inferotemporal cortex** → recognises the whole object ("cat!")

CNNs do the same, layer by layer. This is NOT a coincidence — CNNs were inspired by how the visual brain works (Hubel & Wiesel's Nobel Prize work on cat visual cortex).

### ⚙️ Engineer Analogy
CNNs use **learnable spatial filters** — like Sobel/Gaussian image processing but filter values are LEARNED from data. Key insight: **weight sharing** — the same 3×3 filter slides ALL positions. Only 9 weights describe a pattern for the ENTIRE image, not millions.

**Level:** Intermediate  
**Duration:** ~4 hours  
**Dataset:** CIFAR-10 (via `torchvision` — same as [Kaggle CIFAR-10](https://www.kaggle.com/c/cifar-10))  
**Real-World Use Case:** Image classification for visual inspection, autonomous vehicles, medical imaging

## What You'll Learn
- CNN fundamentals: convolution, pooling, feature maps
- Building CNN architectures with `nn.Conv2d`, `nn.MaxPool2d`, `nn.BatchNorm2d`
- Image transforms with `torchvision.transforms`
- Data augmentation to improve generalisation
- Visualising learned filters and feature maps
- Implementing a ResNet-style skip connection

## CNN Intuition
```
Input Image (32×32×3)
    ↓  Conv2d(3→32, kernel=3)  → 32 feature maps detecting edges
    ↓  ReLU + MaxPool2d(2)     → 16×16×32  (spatial size halved)
    ↓  Conv2d(32→64, kernel=3) → 64 feature maps detecting textures
    ↓  ReLU + MaxPool2d(2)     → 8×8×64
    ↓  Flatten → Linear layers → 10 class scores
```

In [ ]:
# 🧠 Setting up tools and checking hardware for image processing
# ⚙️ torchvision = pretrained models + standard datasets + image transforms built-in
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader

import torchvision
import torchvision.transforms as transforms

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from collections import defaultdict

torch.manual_seed(42)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")

CIFAR10_CLASSES = ('plane','car','bird','cat','deer',
                   'dog','frog','horse','ship','truck')

## 4.1  Data Transforms & Augmentation

### 🧠 Brain Analogy
Your brain recognises a cat whether it's on the left, in dim light, or slightly rotated. Data augmentation gives the model this same robustness by artificially varying training images: RandomCrop = "partial view", RandomHorizontalFlip = "mirror is same object", ColorJitter = "different lighting."

### ⚙️ Engineer Analogy
Augmentation = stochastic transforms applied ONLY during training. Each epoch applies a DIFFERENT random transform → effectively infinite training data. Val transforms MUST be deterministic for fair, reproducible evaluation. Normalise with dataset-specific mean/std → pixels in ~[-2,2] → faster convergence.

**Key insight:** Data augmentation is *applied only during training*, not validation.

In [ ]:
# 🧠 Training: random variations teach robustness. Val: clean consistent images for fair evaluation.
# ⚙️ RandomCrop(32, padding=4): pad to 40×40, crop random 32×32 — simulates different viewpoints
# CIFAR-10 dataset statistics (pre-computed)
CIFAR_MEAN = (0.4914, 0.4822, 0.4465)
CIFAR_STD  = (0.2470, 0.2435, 0.2616)

# ── Training transforms (with augmentation) ───────────────────────────────
train_transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),          # random crop with padding
    transforms.RandomHorizontalFlip(p=0.5),        # mirror 50% of the time
    transforms.ColorJitter(brightness=0.2,
                           contrast=0.2,
                           saturation=0.2),        # color variation
    transforms.ToTensor(),                         # PIL → tensor [0,1]
    transforms.Normalize(CIFAR_MEAN, CIFAR_STD),   # standardize
])

# ── Validation transforms (deterministic) ────────────────────────────────
val_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(CIFAR_MEAN, CIFAR_STD),
])

# ── Download datasets ─────────────────────────────────────────────────────
train_ds = torchvision.datasets.CIFAR10(root='./data', train=True,
                                         transform=train_transform, download=True)
val_ds   = torchvision.datasets.CIFAR10(root='./data', train=False,
                                         transform=val_transform, download=True)

train_loader = DataLoader(train_ds, batch_size=128, shuffle=True,
                          num_workers=2, pin_memory=True)
val_loader   = DataLoader(val_ds,   batch_size=256, shuffle=False,
                          num_workers=2, pin_memory=True)

print(f"Train batches: {len(train_loader)}, Val batches: {len(val_loader)}")

In [ ]:
# 🧠 Look at sample images first — "what are we asking the model to learn to recognise?"
# ⚙️ un-normalise: reverse (x-mean)/std to get pixel values back in [0,1] range for display
# ── Visualise sample images ────────────────────────────────────────────────
def imshow(img_tensor, mean=CIFAR_MEAN, std=CIFAR_STD):
    """Un-normalise and show a tensor image."""
    img = img_tensor.numpy().transpose(1, 2, 0)
    img = img * np.array(std) + np.array(mean)     # un-normalise
    return np.clip(img, 0, 1)

images, labels = next(iter(DataLoader(
    torchvision.datasets.CIFAR10('./data', train=True,
                                  transform=val_transform, download=False),
    batch_size=16, shuffle=True
)))

fig, axes = plt.subplots(2, 8, figsize=(14, 4))
for ax, img, lbl in zip(axes.flatten(), images, labels):
    ax.imshow(imshow(img))
    ax.set_title(CIFAR10_CLASSES[lbl], fontsize=8)
    ax.axis('off')
plt.suptitle("CIFAR-10 Sample Images", fontsize=12)
plt.tight_layout()
plt.show()

## 4.2  CNN Architecture — Step by Step

### 🧠 Brain Analogy
A CNN layer = a team of visual detectors scanning the image: **Conv2d** asks "is MY pattern HERE?", **BatchNorm** normalises responses so no detector dominates, **ReLU** silences detectors that didn't find anything, **MaxPool2d(2)** keeps the STRONGEST detection from each 2×2 patch and shrinks the map by half. MaxPool creates position-invariance: doesn't matter if the edge was at pixel (5,6) or (5,7).

### ⚙️ Engineer Analogy
Conv output size: `H_out = (H_in + 2p - k)/s + 1`. With k=3, p=1, s=1: H_out = H_in. MaxPool(2) halves spatial dims. After 3 pools: 32→16→8→4. Channels grow: 3→32→64→128. Final tensor: 4×4×128=2048 values → flattened for Linear layers.



In [ ]:
# 🧠 CNN = stacked teams of detectors, each layer finding more complex patterns than the last
# ⚙️ ConvBlock = Conv2d→BatchNorm→ReLU: standard building block. MaxPool halves spatial dimensions.
class ConvBlock(nn.Module):
    """Conv → BatchNorm → ReLU (the basic CNN building block)."""

    def __init__(self, in_ch, out_ch, kernel=3, stride=1, padding=1):
        super().__init__()
        self.block = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, kernel, stride, padding, bias=False),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.block(x)


class CIFAR10Net(nn.Module):
    """Custom CNN for CIFAR-10 classification."""

    def __init__(self, num_classes=10):
        super().__init__()

        # Feature extractor
        self.features = nn.Sequential(
            # Block 1: 32×32×3 → 32×32×32
            ConvBlock(3,  32),
            ConvBlock(32, 32),
            nn.MaxPool2d(2),          # → 16×16×32
            nn.Dropout2d(0.2),

            # Block 2: 16×16×32 → 16×16×64
            ConvBlock(32, 64),
            ConvBlock(64, 64),
            nn.MaxPool2d(2),          # → 8×8×64
            nn.Dropout2d(0.3),

            # Block 3: 8×8×64 → 8×8×128
            ConvBlock(64,  128),
            ConvBlock(128, 128),
            nn.MaxPool2d(2),          # → 4×4×128
            nn.Dropout2d(0.4),
        )

        # Classifier head
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(4 * 4 * 128, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x


model = CIFAR10Net().to(device)

# Verify dimensions with a dummy pass
dummy = torch.randn(4, 3, 32, 32).to(device)
print("Output shape:", model(dummy).shape)   # should be (4, 10)
print(f"Parameters: {sum(p.numel() for p in model.parameters()):,}")

## 4.3  ResNet-Style Skip Connection (Intermediate)

### 🧠 Brain Analogy
**Vanishing gradient problem**: error signal weakens as it travels back through many layers — early layers barely update. Skip connections create a **direct shortcut** from later layers back to earlier ones, like an express highway for the error signal. The "+1" in `∂(F(x)+x)/∂x = ∂F/∂x + 1` keeps gradients alive even when `∂F/∂x ≈ 0`.

### ⚙️ Engineer Analogy
Residual block: `out = F(x) + x`. Backprop: `∂out/∂x = ∂F/∂x + 1`. The "+1" prevents vanishing gradients — why ResNet can have 50-150+ layers while plain networks degrade past ~20 layers.

Skip connections solve the **vanishing gradient problem** in deep networks.

In [ ]:
# 🧠 Shortcut = express highway for error signal — bypasses vanishing gradient problem
# ⚙️ out = F(x) + x → gradient has additive identity path, stays strong through depth
class ResidualBlock(nn.Module):
    """Basic residual block: output = F(x) + x"""

    def __init__(self, channels):
        super().__init__()
        self.conv1 = nn.Conv2d(channels, channels, 3, padding=1, bias=False)
        self.bn1   = nn.BatchNorm2d(channels)
        self.conv2 = nn.Conv2d(channels, channels, 3, padding=1, bias=False)
        self.bn2   = nn.BatchNorm2d(channels)
        self.relu  = nn.ReLU(inplace=True)

    def forward(self, x):
        residual = x                         # save input
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out = out + residual                 # skip connection (add input back)
        out = self.relu(out)
        return out


# Quick test
rb = ResidualBlock(64)
x  = torch.randn(2, 64, 16, 16)
print("Residual block output shape:", rb(x).shape)   # (2, 64, 16, 16) — unchanged

## 4.4  Training the CNN

### 🧠 Brain Analogy
OneCycleLR = start with big confident steps (explore broadly), then take tiny careful steps at the end (fine-tune). Like hiking: big strides downhill, careful steps at the bottom. Label smoothing = "don't be 100% sure — some CIFAR-10 photos are genuinely ambiguous."

### ⚙️ Engineer Analogy
`label_smoothing=0.1`: replaces hard labels (0,1) with (0.1, 0.9) — prevents overconfidence. SGD + Nesterov: "looks ahead" before computing gradient. OneCycleLR: LR increases to max then cosine-decays to 0 — explores broadly then refines.



In [ ]:
# 🧠 Training setup: grader (loss), learning strategy (optimizer), schedule (when to slow down)
# ⚙️ OneCycleLR requires total steps upfront: epochs × steps_per_epoch
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)   # label smoothing = regularisation
optimizer = optim.SGD(model.parameters(), lr=0.1,
                      momentum=0.9, weight_decay=5e-4, nesterov=True)
scheduler = optim.lr_scheduler.OneCycleLR(
    optimizer, max_lr=0.1,
    epochs=50, steps_per_epoch=len(train_loader)
)


def run_epoch(model, loader, criterion, optimizer, scheduler, train=True):
    model.train() if train else model.eval()
    total_loss, correct, n = 0.0, 0, 0

    ctx = torch.enable_grad() if train else torch.no_grad()
    with ctx:
        for X, y in loader:
            X, y = X.to(device), y.to(device)

            if train:
                optimizer.zero_grad()

            logits = model(X)
            loss   = criterion(logits, y)

            if train:
                loss.backward()
                optimizer.step()
                scheduler.step()

            total_loss += loss.item() * len(X)
            correct    += (logits.argmax(1) == y).sum().item()
            n          += len(X)

    return total_loss / n, correct / n


EPOCHS = 50
history = defaultdict(list)
best_acc = 0.0

print("Training CNN on CIFAR-10 (this takes a few minutes)...")
for epoch in range(1, EPOCHS + 1):
    tr_loss, tr_acc = run_epoch(model, train_loader, criterion, optimizer, scheduler, train=True)
    vl_loss, vl_acc = run_epoch(model, val_loader,   criterion, optimizer, scheduler, train=False)

    history['tr_loss'].append(tr_loss); history['tr_acc'].append(tr_acc)
    history['vl_loss'].append(vl_loss); history['vl_acc'].append(vl_acc)
    history['lr'].append(optimizer.param_groups[0]['lr'])

    if vl_acc > best_acc:
        best_acc = vl_acc
        torch.save(model.state_dict(), 'cifar10_best.pth')

    if epoch % 10 == 0:
        print(f"Epoch {epoch:3d}/{EPOCHS} | "
              f"Train {tr_acc:.3f} | Val {vl_acc:.3f} | Best {best_acc:.3f}")

print(f"\nBest validation accuracy: {best_acc:.1%}")

In [ ]:
# 🧠 Learning diary: loss down? Accuracy up? Learning rate schedule working as planned?
# ⚙️ Three plots: loss (optimisation target), accuracy (performance), LR (schedule view)
# ── Learning curves ────────────────────────────────────────────────────────
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

axes[0].plot(history['tr_loss'], label='Train'); axes[0].plot(history['vl_loss'], label='Val')
axes[0].set_title('Loss'); axes[0].legend()

axes[1].plot(history['tr_acc'], label='Train'); axes[1].plot(history['vl_acc'], label='Val')
axes[1].set_title('Accuracy'); axes[1].legend()

axes[2].plot(history['lr'])
axes[2].set_title('Learning Rate (OneCycleLR)')

plt.suptitle('CIFAR-10 CNN Training')
plt.tight_layout()
plt.show()

## 4.5  Visualise Learned Filters & Feature Maps

### 🧠 Brain Analogy
After training, we look inside: "What patterns are you looking for?" (filters) and "What do you SEE in this image?" (feature maps). First-layer CNN filters often look like Gabor wavelets — exactly what biological V1 neurons respond to. This convergence of biology and ML engineering is remarkable.

### ⚙️ Engineer Analogy
`register_forward_hook`: non-invasive callback that captures layer output during forward pass — no model modification needed. First conv weights shape: `(32, 3, 3, 3)` = 32 filters, each 3×3 RGB. `register_forward_hook` is the standard tool for mechanistic interpretability.



In [ ]:
# 🧠 "What patterns is the first layer looking for?" — like a V1 neuron preference map
# ⚙️ first_conv.weight.data: (32,3,3,3). Normalise to [0,1] for display.
# ── First conv layer filters ────────────────────────────────────────────
model.load_state_dict(torch.load('cifar10_best.pth', map_location=device))
model.eval()

first_conv = model.features[0].block[0]   # first Conv2d
filters = first_conv.weight.data.cpu()    # shape: (32, 3, 3, 3)

# Normalize each filter to [0,1] for display
filters_norm = (filters - filters.min()) / (filters.max() - filters.min())

fig, axes = plt.subplots(4, 8, figsize=(14, 7))
for i, ax in enumerate(axes.flatten()):
    if i < filters_norm.shape[0]:
        ax.imshow(filters_norm[i].permute(1, 2, 0).numpy())
    ax.axis('off')
plt.suptitle("Learned Filters — First Convolutional Layer (3×3 RGB)", fontsize=12)
plt.tight_layout()
plt.show()

In [ ]:
# 🧠 "What does each detector SEE in THIS image?" — like fMRI of activated brain regions
# ⚙️ Forward hook captures activations; remove hook after use to avoid memory accumulation
# ── Feature maps for one image ────────────────────────────────────────────
sample_img, sample_lbl = val_ds[5]
x = sample_img.unsqueeze(0).to(device)   # add batch dim

# Hook to capture intermediate activations
activations = {}
def make_hook(name):
    def hook(module, inp, output):
        activations[name] = output.detach().cpu()
    return hook

h = model.features[0].block[2].register_forward_hook(make_hook('relu1'))
with torch.no_grad():
    _ = model(x)
h.remove()

feat_maps = activations['relu1'][0]   # shape: (32, 32, 32)

fig, axes = plt.subplots(4, 8, figsize=(14, 7))
for i, ax in enumerate(axes.flatten()):
    if i < feat_maps.shape[0]:
        ax.imshow(feat_maps[i].numpy(), cmap='viridis')
    ax.axis('off')

true_class = CIFAR10_CLASSES[sample_lbl]
plt.suptitle(f"Feature Maps after First ReLU — Image class: '{true_class}'", fontsize=12)
plt.tight_layout()
plt.show()

## 4.6  Per-Class Accuracy Analysis

### 🧠 Brain Analogy
Some objects are harder to recognise — we also find cat vs dog harder than ship vs truck. The model's confusion mirrors human visual confusion. Per-class analysis reveals where to improve: more data? Better augmentation? Harder negative mining?

### ⚙️ Engineer Analogy
Per-class accuracy = `correct[i]/total[i]` for each class. Low accuracy = visually similar to others OR insufficient training examples. Guides data collection and targeted augmentation strategy.



In [ ]:
# 🧠 Where does the model struggle? Which classes does it confuse most?
# ⚙️ Per-class accuracy = diagonal of normalised confusion matrix; compare to mean baseline
class_correct = [0] * 10
class_total   = [0] * 10

model.eval()
with torch.no_grad():
    for X, y in val_loader:
        X, y = X.to(device), y.to(device)
        preds = model(X).argmax(1)
        for pred, true in zip(preds.cpu(), y.cpu()):
            class_correct[true] += (pred == true).item()
            class_total[true]   += 1

print("Per-class accuracy:")
accs = []
for i, cls in enumerate(CIFAR10_CLASSES):
    acc = 100 * class_correct[i] / class_total[i]
    accs.append(acc)
    print(f"  {cls:8s}: {acc:5.1f}%")

plt.figure(figsize=(10, 4))
bars = plt.bar(CIFAR10_CLASSES, accs, color='steelblue', edgecolor='white')
plt.axhline(y=np.mean(accs), color='red', linestyle='--', label=f'Mean: {np.mean(accs):.1f}%')
plt.ylabel('Accuracy (%)')
plt.title('Per-Class Accuracy — CIFAR-10')
plt.legend()
plt.tight_layout()
plt.show()

## 4.7  CNN Architecture Cheatsheet

### 🧠 Brain Analogy
Quick reference for the visual brain's building blocks: Conv = edge/texture scanner, MaxPool = keep strongest signal and shrink, BatchNorm = normalise signals between layers, Dropout2d = silence entire feature maps for robustness, AdaptiveAvgPool = compress to fixed size.

### ⚙️ Engineer Analogy
CNN output size formulas are essential for designing architectures. `AdaptiveAvgPool2d((1,1))` = global average pooling — produces one value per channel regardless of input size, the standard way to make CNNs input-size agnostic.

| Layer | PyTorch | Output Size Formula |
|-------|---------|--------------------|
| Conv2d(in, out, k, s, p) | `nn.Conv2d` | ⌊(H+2p-k)/s⌋+1 |
| MaxPool2d(k, s) | `nn.MaxPool2d` | ⌊(H-k)/s⌋+1 |
| BatchNorm2d(C) | `nn.BatchNorm2d` | Same size |
| Dropout2d(p) | `nn.Dropout2d` | Same size |
| AdaptiveAvgPool2d((H,W)) | `nn.AdaptiveAvgPool2d` | Fixed H×W |

**Common CNN patterns:**
- VGG style: stack 2-3 Conv → MaxPool → Dropout
- ResNet style: 3×3 Conv → BN → ReLU → 3×3 Conv → BN + skip → ReLU
- Inception: parallel 1×1, 3×3, 5×5 convs → concatenate

## Exercises

1. Reduce the model by removing one ConvBlock. How does accuracy change?
2. Try replacing `MaxPool2d` with strided convolutions (`Conv2d(..., stride=2)`).
3. Add the `ResidualBlock` class into `CIFAR10Net` between the conv blocks.
4. Download the [Kaggle CIFAR-10](https://www.kaggle.com/c/cifar-10) dataset and generate predictions for submission.

---
**Next →** [Module 05: Transfer Learning with Pretrained Models](./Module_05_Transfer_Learning.ipynb)